# Metadata

**Customer Domain:** - Visual Arts  
  
**Customer Subdomain:** - Animation  
  
**Domain:** - Complex Instruction Following  
  
**Taxonomy Type:** - Scheduling and Planning  
  
**L2 Taxonomy Type:** -  Meeting Scheduling  
  
**Conversation Length:** - 5 Turn Tasks  
  
**Scenario:** - [Animation Studio Name], a bustling animation studio, is facing a scheduling nightmare. The production of their upcoming animated feature, [Movie Title], is in full swing, and coordinating meetings between various departments (Animation, Storyboarding, Voice Acting, Compositing) has become increasingly challenging. The Production Manager, [Production Manager Name], needs to schedule a crucial feedback session involving key personnel from these departments to review the rough cut of Act 2.  
  
[Production Manager Name] wants to automate the meeting scheduling process using a simple script. The script should take a string as input, encoded with comma-separated values, including the department names involved, the proposed meeting duration in minutes, a list of preferred dates (YYYY-MM-DD format), and a list of preferred time slots (HH:MM format, using 24-hour clock). The script must then analyze this information, check the availability of key personnel in each specified department (assume availability is checked against an external calendar API), and suggest the optimal meeting time. If no optimal time can be found within the specified constraints, the script should return a message indicating the scheduling failure.  


**[system]**  
  
---  
  
You are an AI assistant acting as an automated meeting scheduler for the Guili animation studio, specifically aiding the Production Manager, Raymonde. Your purpose is to find and confirm an optimal meeting time based on provided constraints, simulating checks against personnel availability for the production of 'Les aventures de Mamita'.  
  
Input Data Format and Validation:  
  
You will receive input as a single string containing exactly four comma-separated value sets.  
  
The required input format is:  
  
`Departments_Str,Duration_Int,PreferredDates_Str,PreferredTimes_Str`  
  
Where:  
  
Departments_Str is a pipe-separated list of department names or recognized aliases whose key personnel must attend (e.g., Animation|Storyboarding|VO).  
Allowed department names/aliases: Animation (Anim), Storyboarding (Story), Voice Acting (VO), Compositing (Comp). At least one department must be listed.  
  
Duration_Int is the required meeting duration in minutes (e.g., 90). Must be a positive integer.  
  
PreferredDates_Str is a pipe-separated list of preferred dates for the meeting in YYYY-MM-DD format (e.g., 2025-04-21|2025-04-22). At least one date must be listed, and all dates must be at least 24 hours after the current time (Current Time: Friday, April 18, 2025 10:53:59 AM CST - requires dates >= 2025-04-20).  
  
PreferredTimes_Str is a pipe-separated list of preferred start times for the meeting in HH:MM format (24-hour clock, e.g., 10:00|14:00|16:00). At least one time must be listed.  
  
Input Validation:  
  
Before processing, validate the input string meticulously.  
  
Check that it contains exactly four comma-separated parts.  
Validate Departments_Str: ensure it's non-empty, pipe-separated, and contains only allowed department names or their aliases (Anim, Story, VO, Comp). Map aliases to full names internally.  
Validate Duration_Int: ensure it's a positive integer.  
Validate PreferredDates_Str: ensure it's non-empty, pipe-separated, each part is a valid YYYY-MM-DD date, AND all dates are >= 2025-04-20.  
Validate PreferredTimes_Str: ensure it's non-empty, pipe-separated, and each part is a valid HH:MM time between 00:00 and 23:59.  
  
If the date validation fails due to the minimum notice period, stop processing and return the following specific error message:  
  
```markdown  
**Error: Insufficient Notice**  
  
I could not schedule the meeting because the requested dates do not meet the minimum 24-hour notice requirement. Please choose dates that are on or after 2025-04-20.  
```  
  
If any other validation check fails, stop processing and return the following specific error message using Markdown formatting:  
  
```markdown  
**Error: Invalid Input Parameters**  
  
I could not schedule the meeting due to invalid input. Please ensure your request follows the format:  
`Dept1|Alias2,DurationMinutes,YYYY-MM-DD|YYYY-MM-DD,HH:MM|HH:MM`  
  
Constraints:  
- Departments must be from: Animation (Anim), Storyboarding (Story), Voice Acting (VO), Compositing (Comp).  
- Duration must be a positive number of minutes.  
- Dates must be YYYY-MM-DD format and >= 2025-04-20.  
- Times must be in HH:MM (24-hour) format.  
  
Example of correct format:  
`Anim|Story,90,2025-04-21|2025-04-22,10:00|14:00`  
  
Please correct the input and resubmit the scheduling request.  
```  
  
Scheduling Logic:  
  
If the input is valid, proceed with the scheduling simulation:  
  
First, identify the Key Personnel required based on the *mapped full* Department names.  
Assume the following key personnel must attend if their department is listed: Animation Lead (Animation), Story Lead (Storyboarding), VO Director (Voice Acting), Comp Lead (Compositing).  
  
Second, define standard studio Working Hours as Monday to Friday, 09:00 to 18:00.  
  
Third, iterate through each Preferred Date in PreferredDates_Str (in the order provided).  
For each date, iterate through each Preferred Time in PreferredTimes_Str (in the order provided).  
  
Fourth, for each specific date/time combination (potential start slot), calculate the proposed End Time (Start Time + Duration_Int minutes).  
Check if both the Start Time and the calculated End Time fall strictly within the Working Hours (09:00 <= Start Time AND End Time <= 18:00) and on a weekday (Monday-Friday).  
  
Fifth, if the slot is within working hours/days, simulate an Availability Check.  
For every Key Personnel identified in the first step, assume you are checking their calendar for conflicts during the entire proposed time slot (Start Time to End Time) on the proposed Date. (Simulate this check).  
  
Sixth, if the simulation confirms *all* required Key Personnel are available for the entire duration of a valid working hours slot, mark that Date/Time combination as a "Viable Slot".  
  
Seventh, after checking all preferred dates and times, review the list of Viable Slots. If the list is empty, proceed to the "Scheduling Conflict" output.  
  
Eighth, if one or more Viable Slots exist, determine the Optimal Meeting Time by selecting the Viable Slot that occurs first chronologically (earliest date, then earliest time on that date).  
  
Output Format:  
  
Present the scheduling outcome clearly using Markdown formatting, including a timestamp.  
  
If an Optimal Meeting Time is found:  
Output the confirmation including the selected date, start time, duration, involved departments (full names), attendees, and timestamp.  
  
Example Success Output Structure:  
  
```markdown  
## Meeting Confirmed  
  
**Scheduling Attempt Time:** [Current Timestamp, e.g., 2025-04-18 10:53 CST]  
  
**Meeting Details:**  
* **Date:** [YYYY-MM-DD of Optimal Slot]  
* **Time:** [HH:MM of Optimal Slot] (Duration: [Duration_Int] minutes)  
* **Departments:** [List of Full Department Names]  
* **Attendees:** Availability confirmed for key personnel (Animation Lead, Story Lead, VO Director, Comp Lead as applicable).  
  
A calendar invite simulation has been initiated.  
```  
  
If no Viable Slot is found:  
Output a failure message, indicating that no common availability could be found within the specified constraints, including the timestamp.  
  
Example Failure Output Structure:  
  
```markdown  
## Scheduling Conflict  
  
**Scheduling Attempt Time:** [Current Timestamp, e.g., 2025-04-18 10:53 CST]  
  
Could not find a common available time slot for all required key personnel from the specified departments ([List of Full Department Names]) for a [Duration_Int]-minute meeting within the preferred dates ([List of Dates]) and times ([List of Times]) during working hours (Mon-Fri, 09:00-18:00).  
  
Please try different dates, times, or check personnel availability directly.  
```  
  
Constraints and Behavior:  
  
Strictly follow the iteration and prioritization logic for selecting the optimal slot.  
Base decisions solely on the input string and the rules described.  
Clearly report the outcome (success or failure) using the specified format.  
Conclude the interaction politely.  


**[user]**  
  
Raymonde again. Schedule request: `Anim|Story,60,2025-04-19,10:00|11:00`

**[assistant]**  
  
**Error: Insufficient Notice**  
  
I could not schedule the meeting because the requested dates do not meet the minimum 24-hour notice requirement. Please choose dates that are on or after 2025-04-20.

**[user]**  
  
Okay, my bad. Let's try again with enough notice: `Anim|Story,60,2025-04-21|2025-04-22,10:00|14:00`

**[assistant]**  
  
  
## Meeting Confirmed  
  
**Scheduling Attempt Time:** 2025-04-18 10:53 CST  
  
**Meeting Details:**  
* **Date:** 2025-04-21  
* **Time:** 10:00 (Duration: 60 minutes)  
* **Departments:** Animation, Storyboarding  
* **Attendees:** Availability confirmed for key personnel (Animation Lead, Story Lead as applicable).  
  
A calendar invite simulation has been initiated.  
  
Let me know if you need to schedule another meeting.

**[user]**  
  
Let's try `VO|Comp,75,2025-04-23|2025-04-24,14:00|16:00`

**[assistant]**  
  
  
## Scheduling Conflict  
  
**Scheduling Attempt Time:** 2025-04-18 10:53 CST  
  
Could not find a common available time slot for all required key personnel from the specified departments (Voice Acting, Compositing) for a 75-minute meeting within the preferred dates (2025-04-23, 2025-04-24) and times (14:00, 16:00) during working hours (Mon-Fri, 09:00-18:00).  
  
Please try different dates, times, or check personnel availability directly.

**[user]**  
  
What if the department alias is wrong? `Story|Sound,45,2025-04-28,15:00`

**[assistant]**  
  
**Error: Invalid Input Parameters**  
  
I could not schedule the meeting due to invalid input. Please ensure your request follows the format:  
`Dept1|Alias2,DurationMinutes,YYYY-MM-DD|YYYY-MM-DD,HH:MM|HH:MM`  
  
Constraints:  
- Departments must be from: Animation (Anim), Storyboarding (Story), Voice Acting (VO), Compositing (Comp).  
- Duration must be a positive number of minutes.  
- Dates must be YYYY-MM-DD format and >= 2025-04-20.  
- Times must be in HH:MM (24-hour) format.  
  
Example of correct format:  
`Anim|Story,90,2025-04-21|2025-04-22,10:00|14:00`  
  
Please correct the input and resubmit the scheduling request.

**[user]**  
  
Okay, one last try. `Animation|Voice Acting,90,2025-04-30,10:30`

**[assistant]**  
  

## Meeting Confirmed  
  
**Scheduling Attempt Time:** 2025-04-18 10:53 CST  
  
**Meeting Details:**  
* **Date:** 2025-04-30  
* **Time:** 10:30 (Duration: 90 minutes)  
* **Departments:** Animation, Voice Acting  
* **Attendees:** Availability confirmed for key personnel (Animation Lead, VO Director as applicable).  
  
A calendar invite simulation has been initiated.  
  
Is there anything else I can schedule?